# Hotel Recommendations by using Multimodal + WatsonX.ai 


In this notebook we are going to create a simple pipeline that given some descriptions of you desire the llm will retrive photos of the hotels and based on the photos will sugest top 5 hotels based on your preferences.


First we will just create some premilinar programs to analize how to build the pipeline

# Step 1 - Data Generation
In this part we are going to build the database that contains the pictures and locations and names of the hotels.

In [1]:
import folium
from IPython.display import display, IFrame

In [3]:
import folium

# Function to create a map with a marker at the given latitude and longitude
def create_map(latitude, longitude):
    # Create a map centered at the given coordinates
    map_ = folium.Map(location=[latitude, longitude], zoom_start=12)
    
    # Add a marker at the specified coordinates
    folium.Marker([latitude, longitude], popup='Location').add_to(map_)
    
    # Save the map to an HTML file
    map_.save('map.html')
    return map_



In [4]:
# Example coordinates (latitude, longitude)
latitude = 37.7749
longitude = -122.4194

# Create and display the map
map_ = create_map(latitude, longitude)
map_


In [24]:
import os
import pandas as pd

# 1. Get the Current Directory
current_directory = os.getcwd()
# 2. Construct the Full Path to the CSV File
csv_file_path = os.path.join(current_directory, 'dataset', 'hotel_info.csv')

In [25]:
# Read the CSV file
df_hotels = pd.read_csv(csv_file_path)

In [26]:
df_hotels

,hotel_id,hotel_name,chain_id,latitude,longitude
0,391,Extended Stay America - Fairbanks - Old Airpor...,72,64.83538,-147.82330
1,392,Hilton Hangzhou Qiandao Lake Resort,3,29.60819,119.07290
2,393,Taj Lands End,-1,19.04391,72.81879
3,395,Cambridge Suites Hotel Sydney,-1,46.13663,-60.19551
4,396,Tamanu Beach,14,-18.84213,-159.78794
...,...,...,...,...,...
49995,24274,Hotel Primus Valencia,-1,39.45768,-0.34541
49996,479422,Villa Nautilus,-1,20.20078,-87.45798
49997,354725,Arena Resort,-1,12.67615,101.30842
49998,292795,Indigo Hotel Restaurant & Bar,-1,6.43197,3.43206


In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="hotels")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)

11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland


In [27]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
# Initialize the geolocator
geolocator = Nominatim(user_agent="hotels")
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and location.raw and 'address' in location.raw:
            address = location.raw['address']
            return {
                'city': address.get('city', ''),
                'country': address.get('country', ''),
                'state': address.get('state', ''),
                'county': address.get('county', ''),
                'suburb': address.get('suburb', ''),
                'postcode': address.get('postcode', ''),
                'road': address.get('road', ''),
                'house_number': address.get('house_number', '')
            }
        else:
            return {}
    except GeocoderTimedOut:
        return {}

# Example dataframe
df = df_hotels.head(100)

# Initialize new columns
columns = ['city', 'country', 'state', 'county', 'suburb', 'postcode', 'road', 'house_number']
for col in columns:
    df[col] = ''

C:\Users\rusla\AppData\Local\Temp\ipykernel_6732\582362902.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = ''
C:\Users\rusla\AppData\Local\Temp\ipykernel_6732\582362902.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = ''
C:\Users\rusla\AppData\Local\Temp\ipykernel_6732\582362902.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [28]:
df.head()

,hotel_id,hotel_name,chain_id,latitude,longitude,city,country,state,county,suburb,postcode,road,house_number
0,391,Extended Stay America - Fairbanks - Old Airpor...,72,64.83538,-147.82330,,,,,,,,
1,392,Hilton Hangzhou Qiandao Lake Resort,3,29.60819,119.07290,,,,,,,,
2,393,Taj Lands End,-1,19.04391,72.81879,,,,,,,,
3,395,Cambridge Suites Hotel Sydney,-1,46.13663,-60.19551,,,,,,,,
4,396,Tamanu Beach,14,-18.84213,-159.78794,,,,,,,,


In [30]:
# Populate the new columns
for index, row in df.iterrows():
    info = reverse_geocode(row['latitude'], row['longitude'])
    for key in info:
        df.at[index, key] = info[key]

In [31]:
df

,hotel_id,hotel_name,chain_id,latitude,longitude,city,country,state,county,suburb,postcode,road,house_number
0,391,Extended Stay America - Fairbanks - Old Airpor...,72,64.83538,-147.82330,Fairbanks,United States,Alaska,Fairbanks North Star,,99709,Old Airport Road,4580
1,392,Hilton Hangzhou Qiandao Lake Resort,3,29.60819,119.07290,淳安县,中国,浙江省,,千岛湖镇,,环湖北路,
2,393,Taj Lands End,-1,19.04391,72.81879,Mumbai Suburban,India,Maharashtra,,Zone 3,400035,Byramji Jeejeebhoy Road,
3,395,Cambridge Suites Hotel Sydney,-1,46.13663,-60.19551,,Canada,Nova Scotia,Cape Breton Regional Municipality,,B1P 5G8,Esplanade,356
4,396,Tamanu Beach,14,-18.84213,-159.78794,,Kūki ʻĀirani,Aitutaki,,,,Main Road,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,518,Outrigger Fiji Beach Resort,-1,-18.17764,177.55353,,Viti,Western,Nadroga-Navosa,,,Queens Road,
96,520,Beachfront Hotel,-1,44.29866,-84.73553,,United States,Michigan,Roscommon County,,49629,West Houghton Lake Drive,4761
97,521,Suburban Extended Stay Hotel,-1,37.97763,-87.48288,Evansville,United States,Indiana,Vanderburgh County,,47715,East Indiana Street,5538
98,522,Hotel Ferrari,-1,44.31332,9.32753,,Italia,Liguria,Genova,,16043,Corso Vincenzo De Michiel,57


In [6]:
#df.to_csv('geocoded_hotels.csv', index=False)
import pandas as pd

In [30]:
# Read the CSV file
dfa = pd.read_csv('geocoded_hotels.csv')


In [31]:
dfa.head()

,hotel_id,hotel_name,chain_id,latitude,longitude,city,country,state,county,suburb,postcode,road,house_number
0,391,Extended Stay America - Fairbanks - Old Airpor...,72,64.83538,-147.82330,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
1,392,Hilton Hangzhou Qiandao Lake Resort,3,29.60819,119.07290,淳安县,中国,浙江省,NaN,千岛湖镇,NaN,环湖北路,NaN
2,393,Taj Lands End,-1,19.04391,72.81879,Mumbai Suburban,India,Maharashtra,NaN,Zone 3,400035,Byramji Jeejeebhoy Road,NaN
3,395,Cambridge Suites Hotel Sydney,-1,46.13663,-60.19551,NaN,Canada,Nova Scotia,Cape Breton Regional Municipality,NaN,B1P 5G8,Esplanade,356
4,396,Tamanu Beach,14,-18.84213,-159.78794,NaN,Kūki ʻĀirani,Aitutaki,NaN,NaN,NaN,Main Road,NaN


In [32]:
dfa.columns

Index(['hotel_id', 'hotel_name', 'chain_id', 'latitude', 'longitude', 'city',
       'country', 'state', 'county', 'suburb', 'postcode', 'road',
       'house_number'],
      dtype='object')

In [33]:
dfa.shape

(50000, 13)

In [34]:
import os
import pandas as pd

# 1. Get the Current Directory
current_directory = os.getcwd()

# 2. Construct the Full Path to the CSV Files
path_1 = os.path.join(current_directory, 'dataset', 'test_set.csv')
path_2 = os.path.join(current_directory, 'dataset', 'train_set.csv')

# 3. Load the datasets
test_set = pd.read_csv(path_1)
train_set = pd.read_csv(path_2, header=None)

# 4. Add column names to the train set
columns = ['image_id', 'hotel_id', 'image_url', 'image_source', 'upload_timestamp']
train_set.columns = columns

# 5. Select the specified columns (although they should already match)
test_set_selected = test_set[columns]
train_set_selected = train_set[columns]

# 6. Append/Merge the datasets
merged_dataset = pd.concat([test_set_selected, train_set_selected], ignore_index=True)


In [35]:
merged_dataset

,image_id,hotel_id,image_url,image_source,upload_timestamp
0,2587571,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
1,2587572,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
2,2587573,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
3,2587574,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35
4,2587796,88725,https://traffickcam.com/images/2017/6/20160531...,traffickcam,5/31/16 19:23
...,...,...,...,...,...
1140381,9484584,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28
1140382,9484585,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28
1140383,9484586,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28
1140384,9484587,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28


In [36]:
merged_dataset.columns

Index(['image_id', 'hotel_id', 'image_url', 'image_source',
       'upload_timestamp'],
      dtype='object')

In [37]:
# Ensure 'hotel_id' is a string type for consistent merging (if it's not already)
merged_dataset['hotel_id'] = merged_dataset['hotel_id'].astype(str)
dfa['hotel_id'] = dfa['hotel_id'].astype(str)

In [38]:
# Merge the dataframes on 'hotel_id'
final_merged_dataset = pd.merge(merged_dataset, dfa, on='hotel_id', how='left')


In [39]:


# Assuming final_merged_dataset is already loaded as a dataframe

# Filter rows where both latitude and longitude are not null
filtered_final_merged_dataset = final_merged_dataset.dropna(subset=['latitude', 'longitude'])


In [40]:
filtered_final_merged_dataset

,image_id,hotel_id,image_url,image_source,upload_timestamp,hotel_name,chain_id,latitude,longitude,city,country,state,county,suburb,postcode,road,house_number
0,2587571,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35,Moonrise Hotel,-1,38.65538,-90.29884,Saint Louis,United States,Missouri,NaN,NaN,63112,Delmar Boulevard,6177
1,2587572,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35,Moonrise Hotel,-1,38.65538,-90.29884,Saint Louis,United States,Missouri,NaN,NaN,63112,Delmar Boulevard,6177
2,2587573,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35,Moonrise Hotel,-1,38.65538,-90.29884,Saint Louis,United States,Missouri,NaN,NaN,63112,Delmar Boulevard,6177
3,2587574,79733,https://traffickcam.com/images/2017/4/20160304...,traffickcam,3/4/16 23:35,Moonrise Hotel,-1,38.65538,-90.29884,Saint Louis,United States,Missouri,NaN,NaN,63112,Delmar Boulevard,6177
4,2587796,88725,https://traffickcam.com/images/2017/6/20160531...,traffickcam,5/31/16 19:23,Adam's Mark Buffalo,-1,42.88365,-78.87990,Buffalo,United States,New York,Erie County,NaN,14202,Lower Terrace,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140381,9484584,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28,El Gran Escape,-1,21.21011,-86.71790,NaN,México,Quintana Roo,Isla Mujeres,NaN,77400,Payo Obispo,NaN
1140382,9484585,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28,El Gran Escape,-1,21.21011,-86.71790,NaN,México,Quintana Roo,Isla Mujeres,NaN,77400,Payo Obispo,NaN
1140383,9484586,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28,El Gran Escape,-1,21.21011,-86.71790,NaN,México,Quintana Roo,Isla Mujeres,NaN,77400,Payo Obispo,NaN
1140384,9484587,478252,https://i.travelapi.com/hotels/13000000/129900...,travel_website,2019-12-24 21:10:28,El Gran Escape,-1,21.21011,-86.71790,NaN,México,Quintana Roo,Isla Mujeres,NaN,77400,Payo Obispo,NaN


In [18]:
# Select rows where hotel_id is 391 in dfa
filtered_final_merged_dataset[filtered_final_merged_dataset['hotel_id'] == "391"]

,image_id,hotel_id,image_url,image_source,upload_timestamp,hotel_name,chain_id,latitude,longitude,city,country,state,county,suburb,postcode,road,house_number
60801,6500881,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
60802,6500882,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
60803,6500883,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
60804,6500884,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
60805,6500885,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
60806,6500886,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580
60807,6500887,391,https://i.travelapi.com/hotels/1000000/610000/...,travel_website,2019-12-19 06:44:23,Extended Stay America - Fairbanks - Old Airpor...,72.0,64.83538,-147.8233,Fairbanks,United States,Alaska,Fairbanks North Star,NaN,99709,Old Airport Road,4580


In [41]:
filtered_final_merged_dataset.size

19386562

In [42]:
filtered_final_merged_dataset.to_csv("hotel_multimodal.csv", index=False)  

In [21]:
import pandas as pd
import os

# File Path:
current_directory = os.getcwd()
csv_file_path = os.path.join(current_directory, 'backup' ,'hotel_info.csv')

# Function to read a CSV file from the specified starting index:
def read_csv_from_row(csv_file, start_index=44002, chunksize=1):
    """
    Reads a CSV file from a given start_index, in chunks to optimize memory usage.
    
    Args:
        csv_file (str): Path to the CSV file.
        start_index (int): Row index to start reading from (default is 20000).
        chunksize (int): Number of rows to read at a time (default is 1000).

    Yields:
        pd.DataFrame: A DataFrame containing a chunk of the data.
    """

    with pd.read_csv(csv_file, header=None, skiprows=start_index, chunksize=chunksize) as reader: 
        for chunk in reader:  
            chunk.columns = pd.read_csv(csv_file, nrows=0).columns
            yield chunk  

# Usage:
for df_chunk in read_csv_from_row(csv_file_path):
    # Process each chunk (df_chunk) here 
    print(df_chunk.head())

   hotel_id        hotel_name  chain_id  latitude  longitude
0    113028  HAtel De La Loge        -1  42.69982    2.89401
   hotel_id   hotel_name  chain_id  latitude  longitude
1    113032  Comfort Inn        88  37.36965 -118.39538
   hotel_id         hotel_name  chain_id  latitude  longitude
2    113039  Quality Inn Selma        84  32.43373  -87.02426
   hotel_id                 hotel_name  chain_id  latitude  longitude
3    113059  The Drake Hotel Oak Brook        -1  41.84498  -87.92913
   hotel_id                                     hotel_name  chain_id  \
4    113113  Extended Stay America Houston - The Woodlands        72   

   latitude  longitude  
4  30.13485  -95.44614  
   hotel_id         hotel_name  chain_id  latitude  longitude
5    113118  Econo Lodge North        70  43.60312  -96.71129
   hotel_id                         hotel_name  chain_id  latitude  longitude
6    113135  Hilton Garden Inn Downtown Dallas        87   32.7822  -96.79908
   hotel_id                

In [22]:
df_chunk.head()

,hotel_id,hotel_name,chain_id,latitude,longitude
5998,478252,El Gran Escape,-1,21.21011,-86.7179


In [26]:
import pandas as pd
import os

# File Path:
current_directory = os.getcwd()
csv_file_path = os.path.join(current_directory, 'backup', 'hotel_info.csv')

# Function to read a CSV file from the specified starting index:
def read_csv_from_row(csv_file, start_index=20001):
    """
    Reads a CSV file from a given start_index while keeping the header.
    
    Args:
        csv_file (str): Path to the CSV file.
        start_index (int): Row index to start reading from (default is 20001).

    Returns:
        pd.DataFrame: A DataFrame containing the data from the specified start index.
    """
    return pd.read_csv(csv_file, skiprows=range(1, start_index))

# Usage:
df_chunk = read_csv_from_row(csv_file_path)



In [27]:
df_chunk

,hotel_id,hotel_name,chain_id,latitude,longitude
0,27373,Holiday Inn Express & Suites Seymour,92,38.95901,-85.84648
1,27374,The Pines Inn Lake Placid,-1,44.29271,-73.98920
2,27375,Royal Plaza,-1,50.07825,14.43217
3,27376,Motel 6 - Jacksonville AR,75,34.88638,-92.10823
4,27377,Novum Hotel Eleazar City Center,-1,53.55368,10.00980
...,...,...,...,...,...
29995,24274,Hotel Primus Valencia,-1,39.45768,-0.34541
29996,479422,Villa Nautilus,-1,20.20078,-87.45798
29997,354725,Arena Resort,-1,12.67615,101.30842
29998,292795,Indigo Hotel Restaurant & Bar,-1,6.43197,3.43206
